In [ ]:
%%capture
!pip install timm torchmetrics wandb neurokit2 focal_loss_torch

In [ ]:
# Загрузка датасета
%%capture
!gdown 1NhdUKxIbUUtmaY5avSUoNjPnEjlZfjF5
!unzip "simple_r_peaks.zip"

In [ ]:
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as ptl
import torchmetrics
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from IPython.display import clear_output

from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import models, transforms
import timm
from scipy.signal import resample
import neurokit2 as nk
from focal_loss.focal_loss import FocalLoss

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


device(type='cuda')

In [ ]:
!wandb login 20f90c141c14e3f8e2d80816e21a0960cf446951
import wandb


wandb.init(
    project="aiijc_final",

    config={
        "architecture": "senresnext",
        "dataset": "simple_r_peaks",
    }
)

In [ ]:
data = pd.read_csv('/content/simple_r_peaks/labels.csv', index_col='Unnamed: 0')

In [ ]:
class SignalsDataset(Dataset):
    def __init__(self, labels, path='/'):
        self.x_paths = [labels.iloc[i, 0] for i in range(len(labels))]
        self.labels = [labels.iloc[i, 1] for i in range(len(labels))]
        self.path = path

    def __len__(self):
        return len(self.x_paths)

    def __getitem__(self, idx):
        hr = torch.tensor(np.load(self.path + self.x_paths[idx] + '.npy'))[None, :, :]

        target = self.labels[idx]

        return hr, target

In [ ]:
dataset = SignalsDataset(data, '/content/simple_r_peaks/signals/')
train_data, val_data = random_split(dataset, [0.9, 0.1])

batch_size = 64

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

In [ ]:
model = timm.create_model('gluon_seresnext101_32x4d' , pretrained=True)

model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(in_features=2048, out_features=7, bias=True)

for param in model.parameters():
     param.requires_grad = True

model.to(device)
None

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name gluon_seresnext101_32x4d to current seresnext101_32x4d.gluon_in1k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/196M [00:00<?, ?B/s]

In [ ]:
loss_fn = FocalLoss(gamma=2)

lr = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [ ]:
def train(model, scheduler, optimizer, loss_fn, n_epoch=3, device='cuda'):
    train_losses = []
    val_losses = []
    train_acc = []
    val_acc = []
    val_f1 = []

    max_f1 = 0

    val_accuracy, val_loss, val_f1_score = evaluate(model, val_loader, loss_fn=loss_fn, device=device)
    wandb.log({"F1": val_f1_score, "Acc": val_accuracy, 'loss': val_loss})

    for epoch in range(n_epoch):
        print("Epoch:", epoch+1)

        model = model.train()
        for batch in tqdm(train_loader):
            X_batch, y_batch = batch
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch.float())

            logits = torch.nn.functional.softmax(logits, dim=1)

            loss = loss_fn(logits, y_batch.to(torch.int64))

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            train_losses.append(loss.item())

            model_answers = torch.argmax(logits, dim=1).to(torch.int64)

            train_accuracy = torch.sum(y_batch == model_answers) / len(y_batch)
            train_acc.append(train_accuracy.item())


        model.eval()

        val_accuracy, val_loss, val_f1_score = evaluate(model, val_loader, loss_fn=loss_fn, device=device)
        wandb.log({"F1": val_f1_score, "Acc": val_accuracy, 'loss': val_loss})
        clear_output(wait=True)

        if max_f1 < val_f1_score:
            max_f1 = val_f1_score
            torch.save(model.state_dict(), f'/content/drive/MyDrive/aiijc/models/se_resnext__f1:{val_f1_score:.4f}_5000.pth')

        val_losses.append(val_loss.item())
        val_acc.append(val_accuracy)

        scheduler.step(val_loss)

def evaluate(model, dataloader, loss_fn, device):
    losses = []
    num_correct = 0
    num_elements = 0
    f1 = torchmetrics.F1Score(task='multiclass', num_classes=7)
    f1_score = 0

    model = model.eval()
    for batch in tqdm(dataloader):
        X_batch, y_batch = batch
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).float()

        with torch.no_grad():
            logits = model(X_batch.float())

            logits = torch.nn.functional.softmax(logits, dim=1)

            loss = loss_fn(logits, y_batch.to(torch.int64))
            losses.append(loss.item())

            y_pred = torch.argmax(logits, dim=1).to(torch.int64)

            f1_score += f1(y_pred.cpu(), y_batch.cpu())

            num_elements += len(y_batch)
            num_correct += torch.sum(y_pred == y_batch)

    accuracy = num_correct / num_elements
    f1_score = f1_score / len(dataloader)

    return accuracy.item(), np.mean(losses), f1_score.item()

In [ ]:
train(model=model, scheduler=scheduler, optimizer=optimizer, loss_fn=loss_fn, n_epoch=40, device=device)